In [ ]:
# Data visualize를 위한 imports
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
# GPU 한 개만 할당을 위함
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]="2"

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.datasets import ImageFolder
from torchvision import transforms
from configparser import Interpolation
from saveLoad import save, load
from sklearn.metrics import confusion_matrix
import seaborn as sn
from util import TwoCropTransform, AverageMeter
from util import adjust_learning_rate, warmup_learning_rate
from util import set_optimizer, save_model
from networks.resnet_big import SupConResNet
from losses import SupConLoss

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

pd.set_option('display.max.colwidth', 30)

# 경로 설정하기
original_data_path = '/disk1/colonoscopy_dataset/cropped/' # ADC / HGD / LGD / NOR
base_dir = '/home/sundongk/Multiclass_classification_contrastive'
data_dir = original_data_path
ckpt_dir = os.path.join(base_dir, "checkpoint")
root_dir = '/home/sundongk/dataset/'
# 각 이미지들이 속해 있는 경로
ADC_img_path = '/disk1/colonoscopy_dataset/cropped/ADC/'
HGD_img_path = '/disk1/colonoscopy_dataset/cropped/HGD/'
LGD_img_path = '/disk1/colonoscopy_dataset/cropped/LGD/'
NOR_img_path = '/disk1/colonoscopy_dataset/cropped/NOR/'

In [ ]:
# 가) 전체 파일 수, ADC data 수와 labeled data 수 확인

ADC_file_list = os.listdir(ADC_img_path) # ADC 전체 파일 목록
HGD_file_list = os.listdir(HGD_img_path) # HGD 전체 파일 목록
LGD_file_list = os.listdir(LGD_img_path) # LGD 전체 파일 목록
NOR_file_list = os.listdir(NOR_img_path) # NOR 전체 파일 목록

ADC_data_list = sorted([x for x in ADC_file_list if 'IMG' in x])        # ADC data 파일 목록
ADC_labeled_list = sorted([x for x in ADC_file_list if 'MASK' in x])   # ADC mask 파일 목록

HGD_data_list = sorted([x for x in HGD_file_list if 'IMG' in x])        # HGD data 파일 목록
HGD_labeled_list = sorted([x for x in HGD_file_list if 'MASK' in x])    # HGD mask 파일 목록

LGD_data_list = sorted([x for x in LGD_file_list if 'IMG' in x])        # LGD data 파일 목록
LGD_labeled_list = sorted([x for x in LGD_file_list if 'MASK' in x])    # LGD mask 파일 목록

NOR_data_list = sorted([x for x in NOR_file_list if 'IMG' in x])        # NOR data 파일 목록
NOR_labeled_list = sorted([x for x in NOR_file_list if 'MASK' in x])    # NOR mask 파일 목록

totalNum_list = [len(ADC_file_list), len(HGD_file_list), len(LGD_file_list), len(NOR_file_list)]
totalData_list = [len(ADC_data_list), len(HGD_data_list), len(LGD_data_list), len(NOR_data_list)]
totalMask_list = [len(ADC_labeled_list), len(HGD_labeled_list), len(LGD_labeled_list), len(NOR_labeled_list)]

data = [totalData_list, totalMask_list, totalNum_list]
table = pd.DataFrame(data = data, index = ['data #', 'mask data #', 'Total #'], columns = ['ADC', 'HGD', 'LGD', 'NOR'])

total_img_name_list = [ADC_data_list, HGD_data_list, LGD_data_list, NOR_data_list]
total_label_name_list = [ADC_labeled_list, HGD_labeled_list, LGD_labeled_list, NOR_labeled_list]

print(table)

# Dataset 만들기

- Train, validation, test를 각각 8 : 1 : 1의 비율로 나누어 만든다. 
  - 총 데이터: 3004장 = (2403, 300, 301)
  - Train, validation의 batch size는 32로 하고, test의 batch size는 1로 한다.
- 현재 task는 <font color=yellow>classification이므로 label을 바이너리</font>로 만들어주어야 한다.
  - ADC: 0
  - HGD: 1
  - LGD: 2
  - NOR: 3
- 전처리 사항
  - 256*256으로 resize + Bilinear interpolation
  - random affine (shear=10, scale=(0.8, 1.2))
  - random horizontal flip()
  - 전체 데이터의 평균과 표준편차로 normalize (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

In [ ]:
# 8 : 1 : 1
ADC_train_length = int(len(ADC_data_list)*0.8) # 403
HGD_train_length = int(len(HGD_data_list)*0.8) # 400
LGD_train_length = int(len(LGD_data_list)*0.8) # 800
NOR_train_length = int(len(NOR_data_list)*0.8) # 800 ----- total 2403

ADC_valid_length = int(len(ADC_data_list)*0.1) # 50
HGD_valid_length = int(len(HGD_data_list)*0.1) # 50
LGD_valid_length = int(len(LGD_data_list)*0.1) # 100
NOR_valid_length = int(len(NOR_data_list)*0.1) # 100 ----- total 300

ADC_test_length = len(ADC_data_list) - ADC_train_length - ADC_valid_length # 51
HGD_test_length = len(HGD_data_list) - HGD_train_length - HGD_valid_length # 50
LGD_test_length = len(LGD_data_list) - LGD_train_length - LGD_valid_length # 100
NOR_test_length = len(NOR_data_list) - NOR_train_length - NOR_valid_length # 100 ----- total 301

# Custom dataloader

In [ ]:
# # Dataloader 구현
# class Dataset(Dataset):
#     def __init__(self, data_dir, transform=None, type=None):
#         self.data_dir = data_dir; self.transform = transform; self.type = type

#         lst_label = []; lst_input = []; target_input = []; target_label = []

#         # if type=0, train mode
#         if type == 0:
#             target_input = (ADC_data_list[:ADC_train_length] + HGD_data_list[:HGD_train_length] 
#                                 + LGD_data_list[:LGD_train_length] + NOR_data_list[:NOR_train_length])
#             target_label = [0]*ADC_train_length + [1]*HGD_train_length + [2]*LGD_train_length + [3]*NOR_train_length
        
#         # if type=1, valid mode
#         elif type == 1:
#             target_input = (ADC_data_list[ADC_train_length : ADC_train_length + ADC_valid_length] 
#                                 + HGD_data_list[HGD_train_length : HGD_train_length + HGD_valid_length] 
#                                 + LGD_data_list[LGD_train_length : LGD_train_length + LGD_valid_length] 
#                                 + NOR_data_list[NOR_train_length : NOR_train_length + NOR_valid_length])
#             target_label = [0]*ADC_valid_length + [1]*HGD_valid_length + [2]*LGD_valid_length + [3]*NOR_valid_length

#         # if type=2, test mode
#         else:
#             target_input = (ADC_data_list[ADC_train_length+ADC_valid_length:] 
#                                 + HGD_data_list[HGD_train_length + HGD_valid_length:] 
#                                 + LGD_data_list[LGD_train_length + LGD_valid_length:] 
#                                 + NOR_data_list[NOR_train_length + NOR_valid_length:])
#             target_label = [0]*ADC_test_length + [1]*HGD_test_length + [2]*LGD_test_length + [3]*NOR_test_length

#         self.lst_label = target_label
#         self.lst_input = target_input
    
#     def __len__(self):
#         return len(self.lst_label)
    
#     def __getitem__(self, index):
#         datapath = ''
#         if 'ADC' in self.lst_input[index]: # 0
#             datapath = os.path.join(self.data_dir, 'ADC')
#         elif 'HGD' in self.lst_input[index]: # 1
#             datapath = os.path.join(self.data_dir, 'HGD')
#         elif 'LGD' in self.lst_input[index]: # 2
#             datapath = os.path.join(self.data_dir, 'LGD')
#         elif 'NOR' in self.lst_input[index]: # 3
#             datapath = os.path.join(self.data_dir, 'NOR')
        
#         input = Image.open(os.path.join(datapath, self.lst_input[index])).convert('RGB')
#         if self.transform is not None:
#             input = self.transform(input)

#         data = {'input': input, 'label': self.lst_label[index]}

#         return data

In [ ]:
# # 데이터셋의 평균과 표준편차 구하기
# dataset_train = Dataset(data_dir=original_data_path, transform=transforms.ToTensor(), type=0)
# full_loader = DataLoader(dataset_train, shuffle=False)

# N_CHANNELS = 3
# mean = torch.zeros(3)
# std = torch.zeros(3)

# for item in full_loader:
#     inputs = item['input']
#     for i in range(N_CHANNELS):
#         mean[i] += inputs[:,i,:,:].mean()
#         std[i] += inputs[:,i,:,:].std()
# mean.div_(len(dataset_train)) # tensor([0.5907, 0.3514, 0.2599]) tensor([0.5875, 0.3497, 0.2598])
# std.div_(len(dataset_train)) # tensor([0.2410, 0.1998, 0.1630]) tensor([0.2424, 0.1995, 0.1624])

# Transforms

In [ ]:
train_path = os.path.join(root_dir, 'train')
valid_path = os.path.join(root_dir, 'valid')
test_path = os.path.join(root_dir, 'test')

batch_size = 32; test_batch_size = 1

data_transforms = {
    'train':
    transforms.Compose([
        transforms.RandomResizedCrop(size=64, scale=(0.2, 1.)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.590, 0.351, 0.259],
                                 std=[0.241, 0.199, 0.163])
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((100,100)),
        transforms.ToTensor(),
        # transforms.Normalize(mean=[0.485, 0.456, 0.406],
        #                          std=[0.229, 0.224, 0.225])
        transforms.Normalize(mean=[0.590, 0.351, 0.259],
                                 std=[0.241, 0.199, 0.163])
    ]),
    'test':
    transforms.Compose([
        transforms.Resize((100,100), interpolation=transforms.InterpolationMode.BILINEAR),
        transforms.ToTensor(),
        # transforms.Normalize(mean=[0.485, 0.456, 0.406],
        #                          std=[0.229, 0.224, 0.225])
        transforms.Normalize(mean=[0.590, 0.351, 0.259],
                                 std=[0.241, 0.199, 0.163])
    ])
}

image_datasets = {
    'train': ImageFolder(root_dir=train_path, transform=TwoCropTransform(data_transforms['train']), target_transform=None),
    'validation': ImageFolder(root_dir=valid_path, transform=data_transforms['validation'], target_transform=None),
    'test': ImageFolder(root_dir=test_path, transform=data_transforms['test'], target_transform=None),
}

dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True, num_workers=8),
    'validation': DataLoader(image_datasets['validation'], batch_size=batch_size, shuffle=False, num_workers=8),
    'test': DataLoader(image_datasets['test'], batch_size=test_batch_size, shuffle=False, num_workers=8)
}

# Resnet50

- Supervised contrastive learning

In [ ]:
# from torchvision import models

# model = models.resnet50(pretrained=True).to(device)

# # # freeze
# # for param in model.parameters():
# #     param.requires_grad = False     

# model.fc = nn.Sequential(
#                nn.Linear(2048, 256),
#                nn.ReLU(inplace=True),
#                nn.Linear(256, 4)).to(device)
model = SupConResNet()
criterion = SupConLoss()
lr = 0.0001
optimizer = optim.Adam(model.parameters(), lr = lr)

# 다양한 loss 함수 정의와 학습 함수

In [ ]:
# class FocalLoss(nn.Module):
#     def __init__(self, gamma=0, alpha=None, size_average=True, device='cpu'):
#         super(FocalLoss, self).__init__()
#         """
#         gamma(int) : focusing parameter.
#         alpha(list) : alpha-balanced term.
#         size_average(bool) : whether to apply reduction to the output.
#         """
#         self.gamma = gamma
#         self.alpha = alpha
#         self.size_average = size_average
#         self.device = device

#     def forward(self, input, target):
#         # input : N * C (btach_size, num_class)
#         # target : N (batch_size)

#         CE = F.cross_entropy(input, target, reduction='none')  # -log(pt)
#         pt = torch.exp(-CE)  # pt
#         loss = (1 - pt) ** self.gamma * CE  # -(1-pt)^rlog(pt)

#         if self.alpha is not None:
#             alpha = torch.tensor(self.alpha, dtype=torch.float).to(self.device)
#             # in case that a minority class is not selected when mini-batch sampling
#             if len(self.alpha) != len(torch.unique(target)):
#                 temp = torch.zeros(len(self.alpha)).to(self.device)
#                 temp[torch.unique(target)] = alpha.index_select(0, torch.unique(target))
#                 alpha_t = temp.gather(0, target)
#                 loss = alpha_t * loss
#             else:
#                 alpha_t = alpha.gather(0, target)
#                 loss = alpha_t * loss

#         if self.size_average:
#             loss = torch.mean(loss)

#         return loss

# 학습

In [ ]:
import time

lr = 0.0001
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)
#optimizer = optim.Adam(model.fc.parameters(), lr=0.0001)

checkpoint_path_1 = os.path.join(base_dir, 'chk_loss')
checkpoint_path_2 = os.path.join(base_dir, 'chk_acc')

train_loss = []; val_loss = []; train_acc = []; val_acc = []

def train_model(model, criterion, optimizer, num_epochs=1, st_epoch=0):
    best_loss = 1e10
    best_acc = 0
    for epoch in range(st_epoch+1, num_epochs+1):
        print('Epoch {}/{}'.format(epoch, num_epochs)); print('-' * 10)

        since = time.time()
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'validation']:
            if phase == 'train': 
                model.train()   # Set model to training mode
            else:                
                model.eval()    # Set model to evaluate mode

            running_loss = 0.0; running_corrects = 0

            for batch, item in enumerate(dataloaders[phase], 1):
                if batch % 10 == 0:
                    print('{}/{}'.format(batch, len(dataloaders[phase])))
                inputs = item['input'].to(device)
                labels = item['label'].to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                #FL = FocalLoss(gamma=2, size_average=True)
                #loss = FL(outputs, labels)
                
                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                running_loss += loss.item() * inputs.size(0)

                probs = nn.Softmax(dim=1)(outputs)
                predicts = torch.argmax(outputs, dim=1)

                running_corrects += (predicts==labels).sum().item()
            
            print('{} Accuracy of the model on the {} test images: {}%'.format(phase,
                len(image_datasets[phase]), 100 * running_corrects / len(image_datasets[phase])))

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects / len(image_datasets[phase])

            if phase == 'train':
                train_loss.append(epoch_loss)
                train_acc.append(epoch_acc)
            else:
                val_loss.append(epoch_loss)
                val_acc.append(epoch_acc)
                if epoch_loss < best_loss:
                    print(f"saving best loss model to {checkpoint_path_1}")
                    best_loss = epoch_loss
                    save(ckpt_dir=checkpoint_path_1, model=model, optim=optimizer, epoch=epoch)
                if best_acc < epoch_acc:
                    print(f"saving best acc model to {checkpoint_path_2}")
                    best_acc = epoch_acc
                    save(ckpt_dir=checkpoint_path_2, model=model, optim=optimizer, epoch=epoch)

            time_elapsed = time.time() - since            
            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))

        print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val loss: {:4f}'.format(best_loss))
    
    return model

## 네트워크 학습

In [ ]:
model, optimizer, st_epoch = load(ckpt_dir=checkpoint_path_1, model=model, optim=optimizer)
print(st_epoch)

In [ ]:
model_trained = train_model(model, criterion, optimizer, num_epochs=100, st_epoch=st_epoch)

## Loss 및 Accuracy 저장

In [ ]:
import pickle
# loss 및 acc 정보 저장
loss_dict = {}
loss_dict['train_loss'] = train_loss; loss_dict['val_loss'] = val_loss
loss_dict['train_acc'] = train_acc; loss_dict['val_acc'] = val_acc

# loss_dict 저장
with open('data_dict.pkl','wb') as f:
    pickle.dump(loss_dict,f)

# 시각화 및 평가

In [ ]:
import pickle
# loss_dict 불러오기
loss_info_path = '/home/sundongk/Multiclass_classification_with_gradcam/loss/data_dict.pkl'
with open(loss_info_path,'rb') as f:
    mydict = pickle.load(f)

train_loss = mydict['train_loss']; train_acc = mydict['train_acc']
val_loss = mydict['val_loss']; val_acc = mydict['val_acc']

In [ ]:
# Loss 시각화
plt.figure(figsize=(15, 6)) 
plt.subplot(1,2,1)
plt.ylabel('loss'); plt.xlabel('epoch')
plt.plot(train_loss, 'b', label='train loss')
plt.plot(val_loss, 'g', label='val loss')
plt.legend(loc='best')
plt.show()

In [ ]:
# Acc 시각화
plt.figure(figsize=(15, 6)) 
plt.subplot(1,2,1)
plt.ylabel('accuracy'); plt.xlabel('epoch')
plt.plot(train_acc, 'b', label='train acc')
plt.plot(val_acc, 'g', label='val acc')
plt.legend(loc='best')
plt.show()

In [ ]:
plt.show(); plt.close('all'); plt.clf()
model, optimizer, st_epoch = load(ckpt_dir=checkpoint_path_1, model=model, optim=optimizer)
print(st_epoch)

# Test 데이터셋에 대한 결과

In [ ]:
# Inference
total = correct = 0
model.eval()

with torch.no_grad():
    running_corrects = 0
    
    for batch, item in enumerate(dataloaders['test'], 1):
        # if batch == 202:
        #     break
        inputs = item['input'].to(device)
        labels = item['label'].to(device)

        outputs = model(inputs)

        probs = nn.Softmax(dim=1)(outputs)
        predicts = torch.argmax(outputs, dim=1)

        total += len(labels)
        correct += (predicts == labels).sum().item()
        # print(probs, predicts, labels)
        # # 불러온 이미지 시각화
        # plt.figure(figsize=(12,12))
        # input_image_example = plt.subplot(1,3,1)
        # input_image_example.set_title(f'ADC: {100*pred_probs[0][3]:.2f}, HGD: {100*pred_probs[0][2]:.2f}, LGD: {100*pred_probs[0][1]:.2f}, NOR: {100*pred_probs[0][0]:.2f}')

        # plt.imshow(to_pil_image(changeTensor(inputs)))
        # print(outputs, labels, pred_probs)
        # print(f'NOR: {100*pred_probs[0][0]:.2f}, LGD: {100*pred_probs[0][1]:.2f}, HGD: {100*pred_probs[0][2]:.2f}, ADC: {100*pred_probs[0][3]:.2f}, ')
        # plt.show()
        # plt.close('all')
        # plt.clf()
    print('Test Accuracy of the model on the {} test images: {}%'.format(total, 100 * correct / total))

        

In [ ]:
print(model)

# Confusion matrix

In [ ]:
# ADC: 51, HGD:50, LGD:100, NOR:100
y_pred = []
y_true = []

# Inference
model.eval()
with torch.no_grad():
    for batch, item in enumerate(dataloaders['test'], 1):
        inputs = item['input'].to(device)
        labels = item['label'].to(device)

        outputs = model(inputs)
        outputs = (torch.max(torch.exp(outputs), 1)[1]).data.cpu().numpy()
        y_pred.extend(outputs) # save prediction

        labels = labels.data.cpu().numpy()
        y_true.extend(labels) # save ground truth

class_names = ('ADC', 'HGD', 'LGD', 'NOR')

# Build confusion matrix
cf_matrix = confusion_matrix(y_true, y_pred)
# Create pandas dataframe
dataframe = pd.DataFrame(cf_matrix, index=class_names, columns=class_names)

sn.heatmap(dataframe, annot=True, cbar=None,cmap="YlGnBu",fmt="d")
plt.title("Confusion Matrix"), plt.tight_layout()
 
plt.ylabel("True Class"), 
plt.xlabel("Predicted Class")
plt.show()

In [ ]:
from sklearn.metrics import classification_report
from pprint import pprint as pp

pp(classification_report(y_true, y_pred))

# Grad-CAM

In [ ]:
import cv2 

class FeatureExtractor():
    """ Class for extracting activations and 
    registering gradients from targetted intermediate layers """

    def __init__(self, model, target_layers):
        self.model = model
        self.target_layers = target_layers
        self.gradients = []

    def save_gradient(self, grad):
        self.gradients.append(grad)

    def __call__(self, x):
        outputs = []
        self.gradients = []
        for name, module in self.model._modules.items():
            x = module(x)
            if name in self.target_layers:
                x.register_hook(self.save_gradient)
                outputs += [x]
        return outputs, x


class ModelOutputs():
    """ Class for making a forward pass, and getting:
    1. The network output.
    2. Activations from intermeddiate targetted layers.
    3. Gradients from intermeddiate targetted layers. """

    def __init__(self, model, feature_module, target_layers):
        self.model = model
        self.feature_module = feature_module
        self.feature_extractor = FeatureExtractor(self.feature_module, target_layers)

    def get_gradients(self):
        return self.feature_extractor.gradients

    def __call__(self, x):
        target_activations = []
        #print(x.shape)
        for name, module in self.model._modules.items():
            if module == self.feature_module:
                target_activations, x = self.feature_extractor(x)
            elif "avgpool" in name.lower():
                x = module(x)
                x = x.view(x.size(0),-1)
            else:
                x = module(x)
        
        return target_activations, x


def preprocess_image(img):
    means = [0.485, 0.456, 0.406]
    stds = [0.229, 0.224, 0.225]

    preprocessed_img = img.copy()[:, :, ::-1]
    for i in range(3):
        preprocessed_img[:, :, i] = preprocessed_img[:, :, i] - means[i]
        preprocessed_img[:, :, i] = preprocessed_img[:, :, i] / stds[i]
    preprocessed_img = \
        np.ascontiguousarray(np.transpose(preprocessed_img, (2, 0, 1)))
    preprocessed_img = torch.from_numpy(preprocessed_img)
    preprocessed_img.unsqueeze_(0)
    input = preprocessed_img.requires_grad_(True)
    return input


def show_cam_on_image(img, mask):
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    cam = heatmap + np.float32(img)
    cam = cam / np.max(cam)
    os.path.join(base_dir, "cam.jpg")
    #cv2.imwrite("cam.jpg", np.uint8(255 * cam))
    cv2.imwrite(os.path.join(base_dir, "cam.jpg"), np.uint8(255 * cam))

class GradCam:
    def __init__(self, model, feature_module, target_layer_names, use_cuda):
        self.model = model
        self.feature_module = feature_module
        self.model.eval()
        self.cuda = use_cuda
        if self.cuda:
            self.model = model.cuda()

        self.extractor = ModelOutputs(self.model, self.feature_module, target_layer_names)

    def forward(self, input):
        return self.model(input)

    def __call__(self, input, index=None):
        if self.cuda:
            features, output = self.extractor(input.cuda())#[1, 2048, 7, 7], [1, 1000]
        else:
            features, output = self.extractor(input)


        if index == None:
            index = np.argmax(output.cpu().data.numpy())
        #print(index)

        one_hot = np.zeros((1, output.size()[-1]), dtype=np.float32)#1,1000
        one_hot[0][index] = 1
        one_hot = torch.from_numpy(one_hot).requires_grad_(True)
        if self.cuda:
            one_hot = torch.sum(one_hot.cuda() * output)
            print(one_hot)
        else:
            one_hot = torch.sum(one_hot * output)

        self.feature_module.zero_grad()
        self.model.zero_grad()
        one_hot.backward(retain_graph=True)

        grads_val = self.extractor.get_gradients()[-1].cpu().data.numpy()#1, 2048, 7, 7
        target = features[-1]#1,2048,7,7
        target = target.cpu().data.numpy()[0, :]#2048, 7, 7

        weights = np.mean(grads_val, axis=(2, 3))[0, :]#2048
        cam = np.zeros(target.shape[1:], dtype=np.float32)

        for i, w in enumerate(weights):#w:weight,target:feature
            cam += w * target[i, :, :]

        cam = np.maximum(cam, 0)#7,7
        cam = cv2.resize(cam, input.shape[2:])#224,224
        cam = cam - np.min(cam)
        cam = cam / np.max(cam)
        return cam

In [ ]:
# ADC_img_path = '/disk1/colonoscopy_dataset/cropped/ADC/'
# ADC_file_list = os.listdir(ADC_img_path) # ADC 전체 파일 목록
# ADC_data_list = sorted([x for x in ADC_file_list if 'IMG' in x])        # ADC data 파일 목록

grad_cam = GradCam(model=model, feature_module=model.layer4, target_layer_names=["2"], use_cuda=True)
# Inference
total = correct = 0
model.eval()

#with torch.no_grad():
running_corrects = 0
for item in ADC_data_list:
    path = os.path.join(ADC_img_path, item)
    img = cv2.imread(path, 1)
    img = np.float32(cv2.resize(img, (256, 256))) / 255
    input = preprocess_image(img)
    
    target_index = None
    mask = grad_cam(input, target_index)

    cam = show_cam_on_image(img, mask)